In [7]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

In [18]:
end = dt.datetime(2020, 1, 1)
start = dt.datetime(2016, 1, 1)

df = web.DataReader("TSLA", 'yahoo', start, end)

df = df.reset_index()

nomes = {'Close': 'Close 0',
         'Open': 'Open 0'}

df = df.rename(columns = nomes)

df['Date2'] = df['Date'].astype(str).tolist()
df['Ano'] = df['Date2'].str.split('-').str[0]
df['Mes'] = df['Date2'].str.split('-').str[1]
df['Dia'] = df['Date2'].str.split('-').str[2]

dict_day = { 0: 'Segunda', 1: 'Terca', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta'}

dia = []
for wd in df['Date'].dt.dayofweek:
    dia.append(dict_day[wd])

df['Dia_semana'] = dia

for i in range(1, 6):
    df['Close '+str(i)] = df['Close '+str(i-1)].shift(1)
    df['Open '+str(i)] = df['Open '+str(i-1)].shift(1)

df = df.drop(columns=['Date','Date2','High','Low','Volume','Adj Close'])
        
df.rename(columns={'Close 0':'Target'}, inplace=True)
df.dropna(inplace=True)

In [19]:
df.head()

,Open 0,Target,Ano,Mes,Dia,Dia_semana,Close 1,Open 1,Close 2,Open 2,Close 3,Open 3,Close 4,Open 4,Close 5,Open 5
5,42.801998,41.570000,2016,01,11,Segunda,42.200001,43.571999,43.130001,42.838001,43.807999,44.000000,44.686001,45.271999,44.681999,46.144001
6,42.320000,41.993999,2016,01,12,Terca,41.570000,42.801998,42.200001,43.571999,43.130001,42.838001,43.807999,44.000000,44.686001,45.271999
7,42.402000,40.062000,2016,01,13,Quarta,41.993999,42.320000,41.570000,42.801998,42.200001,43.571999,43.130001,42.838001,43.807999,44.000000
8,40.442001,41.236000,2016,01,14,Quinta,40.062000,42.402000,41.993999,42.320000,41.570000,42.801998,42.200001,43.571999,43.130001,42.838001
9,39.793999,40.998001,2016,01,15,Sexta,41.236000,40.442001,40.062000,42.402000,41.993999,42.320000,41.570000,42.801998,42.200001,43.571999


In [20]:
#Classificar em períodos 
# dia 1 - dia 7 - periodo_1
# dia 8 - dia 14 - periodo_2
# dia 15 - dia 21 - periodo_3
# dia 21 - fim - periodo_4

df['Periodo'] = np.where(df['Dia'].astype(int)<8, 1, 
                        np.where(df['Dia'].astype(int)<15,2,
                        np.where(df['Dia'].astype(int)<21,3,4)))

In [21]:
import sweetviz as sv

report = sv.analyze(df, 'Target')
report.show_html(filepath='REPORT_TSLA.html')



Report REPORT_TSLA.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [52]:
df['Anos'] = df['Ano']
df['Meses'] = df['Mes']
df['Dias'] = df['Dia']
df['Dias_semana'] = df['Dia_semana']
df['Periodos'] = df['Periodo']
df = pd.get_dummies(df, columns=['Ano','Mes','Dia','Dia_semana','Periodo'])
df.head()

,Open 0,Target,Close 1,Open 1,Close-Open_1,Close 2,Open 2,Close-Open_2,Close 3,Open 3,...,Dia_31,Dia_semana_Quarta,Dia_semana_Quinta,Dia_semana_Segunda,Dia_semana_Sexta,Dia_semana_Terca,Periodo_1,Periodo_2,Periodo_3,Periodo_4
5,97.910004,97.510002,97.330002,99.879997,-2.549995,97.919998,100.500000,-2.580002,102.970001,101.129997,...,0,0,0,1,0,0,0,1,0,0
6,99.000000,99.370003,97.510002,97.910004,-0.400002,97.330002,99.879997,-2.549995,97.919998,100.500000,...,0,0,0,0,0,1,0,1,0,0
7,100.580002,95.440002,99.370003,99.000000,0.370003,97.510002,97.910004,-0.400002,97.330002,99.879997,...,0,1,0,0,0,0,0,1,0,0
8,95.849998,98.370003,95.440002,100.580002,-5.139999,99.370003,99.000000,0.370003,97.510002,97.910004,...,0,0,1,0,0,0,0,1,0,0
9,93.980003,94.970001,98.370003,95.849998,2.520004,95.440002,100.580002,-5.139999,99.370003,99.000000,...,0,0,0,0,1,0,0,0,1,0
